In [9]:
#Load in csv file from SQl query
import pandas as pd
df = pd.read_csv('PU_proj_pitcherswithnewteams.csv')
df

,playerID,yearID,teamID,stint,InningsPitched,ERA,FIP
0,abadfe01,2015,OAK,1,47.6667,4.15,3.9692
1,abadfe01,2016,MIN,1,34.0000,2.65,3.2980
2,abadfe01,2016,BOS,2,12.6667,6.39,3.8842
3,abadfe01,2017,BOS,1,43.6667,3.30,3.3527
4,abadfe01,2017,BOS,1,43.6667,3.30,3.3527
...,...,...,...,...,...,...,...
3438,zimmejo02,2017,DET,1,160.0000,6.08,3.8313
3439,zimmejo02,2018,DET,1,131.3333,4.52,3.7584
3440,zimmejo02,2019,DET,1,112.0000,6.91,3.6702
3441,zychto01,2016,SEA,1,13.6667,3.29,2.9073


In [10]:
#See how many distinct players are in table
unique_players = set(df['playerID'])
len(unique_players) 

1076

In [11]:
#Find how many of the players are on the rays, store in a list
playerIDs = []
for i in range(df.shape[0]):
    team = df['teamID'][i]
    if team == 'TBA':
        player_id = df['playerID'][i]
        playerIDs.append(player_id)
rays_players = set(playerIDs)
len(set(rays_players)), len(rays_players) #make sure each player is different

(64, 64)

In [12]:
#Create new filtered dataframe where each player has been on the Rays
filtered_df = df[df['playerID'].isin(rays_players)]
filtered_df.head()

,playerID,yearID,teamID,stint,InningsPitched,ERA,FIP
59,alvarjo03,2018,TBA,1,64.0000,2.39,2.8875
60,alvarjo03,2019,TBA,1,30.0000,4.80,3.5222
83,andrima01,2016,TBA,1,127.6667,4.37,3.4037
84,andrima01,2017,TBA,1,86.0000,4.50,3.7426
85,andrima01,2018,TBA,1,59.6667,4.07,3.3508


In [13]:
unique = set(filtered_df['playerID'])
len(unique)

64

Next Step: Filter dataFrame to show players who were on another team and then joined the Rays ('TBA') immediately after. In other words, now looking for players who had one stint with a not Rays team and then immediately switched to the Rays. Thus, each player that meets these conditions should have two consecutive rows in new dataframe.

General strategy:
-Find the players and rows where a player has a stint with a team that is not the Rays ('TBA').
-Check if their very next stint is with the Rays.
-If it is, save the two rows for displaying later in the dataframe


In [14]:
# Initialize an empty list to store indexes of the rows we want in new df
result = []

# Loop through each unique rays pitcher
for player in rays_players:
    # Extract data for the current player
    player_data = filtered_df[filtered_df['playerID'] == player]
    
    # Extract the list of team IDs for the current player
    team_ids = player_data['teamID'].tolist()

    # Loop through the team IDs to see sequence of teams for players
    for i in range(len(team_ids) - 1):
        # Check if the current team is not Rays and the next entry is Rays 
        if team_ids[i] != 'TBA' and team_ids[i + 1] == 'TBA':
            # Append the consecutive stint to the result list 
            result.append(player_data.iloc[i:i + 2])
            
# Concatenate the filtered results into a new DataFrame
filtered_df = pd.concat(result)

# Print the filtered DataFrame
filtered_df.head()

,playerID,yearID,teamID,stint,InningsPitched,ERA,FIP
3298,weberry01,2017,SEA,1,3.6667,2.45,3.2000
3299,weberry01,2018,TBA,1,5.3333,5.06,3.4500
2921,slegeaa01,2018,MIN,1,13.6667,5.27,4.0049
2922,slegeaa01,2019,TBA,1,3.0000,3.00,4.6444
1556,jennida01,2017,CHA,1,44.3333,3.45,3.6436


In [15]:
#Appending full names column to df
#Dictionary to map player IDs to full names
player_id_to_full_name = {
    'cedenxa01': 'Alex Cedeno',
    'cishest01': 'Steven Cishek',
    'diazju03': 'Junior Diaz',
    'drakeol01': 'Oliver Drake',
    'evelada01': 'David Eveland',
    'farquda01': 'Danny Farquhar',
    'fontwi01': 'Wilking Font',
    'glasnty01': 'Tyler Glasnow',
    'hunteto02': 'Tommy Hunter',
    'jennida01': 'Dan Jennings',
    'jepseke01': 'Kevin Jepsen',
    'milneho01': 'Hoby Milner',
    'mortoch02': 'Charlie Morton',
    'nunovi01': 'Vidal Nuno',
    'paganem01': 'Emilio Pagan',
    'richatr01': 'Tanner Roark',
    'roech01': 'Chris Roe',
    'romose01': 'Sergio Romo',
    'sadleca02': 'Casey Sadler',
    'slegeaa01': 'A. J. Slegers',
    'webbry01': 'Ryan Weber',
    'weberry01': 'Ryan Weathers',
    'whitlch01': 'Chase Whitley'
}

# Add a new column to filtered_df with the full names based on player IDs
filtered_df['Full_Name'] = filtered_df['playerID'].map(player_id_to_full_name)
filtered_df['last_name'] = filtered_df['Full_Name'].str.split().str[-1]
# Move full name column over 
new_column_order = ['playerID', 'last_name', 'Full_Name', 'yearID', 'teamID', 'stint', 'InningsPitched', 'ERA', 'FIP']
filtered_df = filtered_df[new_column_order]
filtered_df.head()

,playerID,last_name,Full_Name,yearID,teamID,stint,InningsPitched,ERA,FIP
3298,weberry01,Weathers,Ryan Weathers,2017,SEA,1,3.6667,2.45,3.2000
3299,weberry01,Weathers,Ryan Weathers,2018,TBA,1,5.3333,5.06,3.4500
2921,slegeaa01,Slegers,A. J. Slegers,2018,MIN,1,13.6667,5.27,4.0049
2922,slegeaa01,Slegers,A. J. Slegers,2019,TBA,1,3.0000,3.00,4.6444
1556,jennida01,Jennings,Dan Jennings,2017,CHA,1,44.3333,3.45,3.6436


In [20]:
# Calculate the difference in ERA: NotRays_ERA - Rays_ERA 
ERA_differences = []
FIP_differences = []
rays_rows = range(1, filtered_df.shape[0], 2) #each row corresponds to one pitcher and their stats on the Rays (every other)
for i in rays_rows:
    ERA_diff = filtered_df['ERA'].iloc[i-1] - filtered_df.iloc[i]['ERA']
    ERA_differences.append(-ERA_diff)
    FIP_diff = filtered_df['FIP'].iloc[i-1] - filtered_df['FIP'].iloc[i]
    FIP_differences.append(-FIP_diff)
    
#Make a new dataframe, add on ERA_differences and FIP_differences columns
Rays_df = filtered_df[filtered_df['teamID'] == 'TBA']
Rays_df['ERA_diff'] = ERA_differences
Rays_df['FIP_diff'] = FIP_differences
Rays_df.head()

C:\Users\Sean Salvador\AppData\Local\Temp\ipykernel_25920\1610247550.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rays_df['ERA_diff'] = ERA_differences
C:\Users\Sean Salvador\AppData\Local\Temp\ipykernel_25920\1610247550.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rays_df['FIP_diff'] = FIP_differences


,playerID,last_name,Full_Name,yearID,teamID,stint,InningsPitched,ERA,FIP,ERA_diff,FIP_diff
3299,weberry01,Weathers,Ryan Weathers,2018,TBA,1,5.3333,5.06,3.4500,2.61,0.2500
2922,slegeaa01,Slegers,A. J. Slegers,2019,TBA,1,3.0000,3.00,4.6444,-2.27,0.6395
1557,jennida01,Jennings,Dan Jennings,2017,TBA,2,18.3333,3.44,3.8545,-0.01,0.2109
754,diazju03,Diaz,Junior Diaz,2017,TBA,1,30.0000,5.70,3.6556,2.56,-0.2188
2251,nunovi01,Nuno,Vidal Nuno,2018,TBA,1,33.0000,1.64,3.5737,-8.79,-1.7854


In [21]:
#Sort by FIP diff
#Just going to focus on the 5 pitchers with the biggest improvement in FIP
sorted_df = Rays_df.sort_values(by='FIP_diff')

#Note that ERA also significantly improved with FIP improvement
top5_Rays = sorted_df.iloc[0:5, ]
top5_Rays

,playerID,last_name,Full_Name,yearID,teamID,stint,InningsPitched,ERA,FIP,ERA_diff,FIP_diff
993,fontwi01,Font,Wilking Font,2018,TBA,3,27.0000,1.67,3.4346,-13.18,-2.7154
2251,nunovi01,Nuno,Vidal Nuno,2018,TBA,1,33.0000,1.64,3.5737,-8.79,-1.7854
468,cedenxa01,Cedeno,Alex Cedeno,2015,TBA,2,43.0000,2.09,3.1147,-3.91,-1.3075
2690,romose01,Romo,Sergio Romo,2017,TBA,2,30.6667,1.47,3.1022,-4.65,-0.9645
541,cishest01,Cishek,Steven Cishek,2017,TBA,2,24.6667,1.09,2.7811,-2.06,-0.9189


In [22]:
top5_Rays.dtypes

playerID           object
last_name          object
Full_Name          object
yearID              int64
teamID             object
stint               int64
InningsPitched    float64
ERA               float64
FIP               float64
ERA_diff          float64
FIP_diff          float64
dtype: object

In [23]:
#Now Import Data From Statcast to Get Pitch Usage Information On These Pitchers
df2_all = pd.read_csv('Statcast Pitch Metrics V2.csv')
df2_all.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'p_opp_batting_avg',
       'xba', 'xslg', 'woba', 'xwoba', 'xobp', 'xiso', 'exit_velocity_avg',
       'launch_angle_avg', 'barrel_batted_rate', 'pitch_hand', 'n', 'ff_pct',
       'ff_avg_speed', 'ff_avg_spin', 'ff_avg_break_x', 'ff_avg_break_z',
       'ff_avg_break', 'ff_range_speed', 'sl_pct', 'sl_avg_speed',
       'sl_avg_spin', 'sl_avg_break_x', 'sl_avg_break_z', 'sl_avg_break',
       'sl_range_speed', 'ch_pct', 'ch_avg_speed', 'ch_avg_spin',
       'ch_avg_break_x', 'ch_avg_break_z', 'ch_avg_break', 'ch_range_speed',
       'cu_pct', 'cu_avg_speed', 'cu_avg_spin', 'cu_avg_break_x',
       'cu_avg_break_z', 'cu_avg_break', 'cu_range_speed', 'si_pct',
       'si_avg_speed', 'si_avg_spin', 'si_avg_break_x', 'si_avg_break_z',
       'si_avg_break', 'si_range_speed', 'fc_pct', 'fc_avg_speed',
       'fc_avg_spin', 'fc_avg_break_x', 'fc_avg_break_z', 'fc_avg_break',
       'fc_range_speed', 'fs_pct', 'fs_avg_speed', 'fs_avg_sp

In [27]:
#Save a table with movement profiles for potential later use
df2_mvmt_profs = df2_all[['last_name', ' first_name', 'year', 'pitch_hand', 
                                          'ff_pct', 'ff_avg_speed','ff_avg_break_x', 'ff_avg_break_z',
                                          'sl_pct', 'sl_avg_speed', 'sl_avg_break_x', 'sl_avg_break_z', 
                                          'ch_pct', 'ch_avg_speed', 'ch_avg_break_x', 'ch_avg_break_z', 
                                          'cu_pct', 'cu_avg_speed', 'cu_avg_break_x', 'cu_avg_break_z', 
                                          'si_pct','si_avg_speed', 'si_avg_break_x', 'si_avg_break_z',
                                          'fc_pct', 'fc_avg_speed','fc_avg_break_x', 'fc_avg_break_z', 
                                          'fs_pct', 'fs_avg_speed',  'fs_avg_break_x', 'fs_avg_break_z', 
                                          'kn_pct', 'kn_avg_speed', 'kn_avg_break_x', 'kn_avg_break_z']]
#make a table with just pitch usage percentages
df2_usages = df2_mvmt_profs[['last_name', ' first_name', 'year', 'pitch_hand', 'ff_pct', 
                            'sl_pct', 'ch_pct','cu_pct', 'si_pct', 'fc_pct', 'fs_pct', 'kn_pct']]
df2_usages

,last_name,first_name,year,pitch_hand,ff_pct,sl_pct,ch_pct,cu_pct,si_pct,fc_pct,fs_pct,kn_pct
0,Colon,Bartolo,2015,R,29.1,9.7,7.4,0.3,53.5,NaN,NaN,NaN
1,Hawkins,LaTroy,2015,R,73.8,16.1,8.0,0.5,1.6,NaN,NaN,NaN
2,Chen,Bruce,2015,L,0.8,31.4,13.6,9.3,44.9,NaN,NaN,NaN
3,Wolf,Randy,2015,L,15.7,24.6,11.1,15.7,33.0,NaN,NaN,NaN
4,Marquis,Jason,2015,R,NaN,19.4,21.2,NaN,59.4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3646,Baez,Michel,2019,R,58.7,5.2,33.2,2.9,NaN,NaN,NaN,NaN
3647,Hirano,Yoshihisa,2019,R,48.0,0.3,NaN,NaN,NaN,NaN,51.7,NaN
3648,Karinchak,James,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3649,Margevicius,Nick,2019,L,54.1,25.3,7.2,13.5,NaN,NaN,NaN,NaN


In [28]:
#Simple Data Validation: Replace NaN with 0 
df2_usages = df2_usages.fillna(0)
df2_usages.dtypes

last_name       object
 first_name     object
year             int64
pitch_hand      object
ff_pct         float64
sl_pct         float64
ch_pct         float64
cu_pct         float64
si_pct         float64
fc_pct         float64
fs_pct         float64
kn_pct         float64
dtype: object

In [29]:
#Join df2_usages with top5_Rays:
merged_df = pd.merge(df2_usages, top5_Rays, left_on=['last_name', 'year'], right_on = ['last_name', 'yearID'], how='inner')
merged_df = merged_df.drop(columns = ['playerID', 'Full_Name', 'yearID', 'stint'])
merged_df = merged_df.sort_values(by='FIP_diff')
merged_df

,last_name,first_name,year,pitch_hand,ff_pct,sl_pct,ch_pct,cu_pct,si_pct,fc_pct,fs_pct,kn_pct,teamID,InningsPitched,ERA,FIP,ERA_diff,FIP_diff
3,Font,Wilmer,2018,R,59.1,9.3,0.0,17.0,6.4,0.0,8.2,0.0,TBA,27.0000,1.67,3.4346,-13.18,-2.7154
4,Nuno,Vidal,2018,L,17.4,67.0,9.2,6.4,0.0,0.0,0.0,0.0,TBA,33.0000,1.64,3.5737,-8.79,-1.7854
0,Cedeno,Xavier,2015,L,44.0,0.0,1.6,52.5,1.8,0.0,0.0,0.0,TBA,43.0000,2.09,3.1147,-3.91,-1.3075
1,Romo,Sergio,2017,R,18.7,58.4,7.2,0.0,15.7,0.0,0.0,0.0,TBA,30.6667,1.47,3.1022,-4.65,-0.9645
2,Cishek,Steve,2017,R,2.2,48.7,1.4,0.0,47.7,0.0,0.0,0.0,TBA,24.6667,1.09,2.7811,-2.06,-0.9189


In [30]:
merged_df.columns

Index(['last_name', ' first_name', 'year', 'pitch_hand', 'ff_pct', 'sl_pct',
       'ch_pct', 'cu_pct', 'si_pct', 'fc_pct', 'fs_pct', 'kn_pct', 'teamID',
       'InningsPitched', 'ERA', 'FIP', 'ERA_diff', 'FIP_diff'],
      dtype='object')

In [31]:
#Add new row into df with data for previous stint of each pitcher
#Pitch briefly for WSH in 2014 and 2015 (14 games total)
Cedeno_2 = ['Cedeno', 'Xavier', 2014, 'L', 59.8, 0, 1.8, 32.1, 6.3, 0, 0, 0, 'WSH', 10, 4.50, 7.46, 0, 0 ]
Romo_2 = ['Romo', 'Sergio', 2016, 'R', 7.9, 63.5, 7.3, 0, 21.2, 0,0,0, 'SFG', 30.2, 6.12, 4.07, 0,0 ]
Cishek_2 = ['Cishek', 'Steve', 2016, 'R', 1.5, 50.1, 1.0, 0, 47.3, 0,0,0, 'SEA', 64, 1.09+2.06, 2.78+.92, 0,0]
Font_2 = ['Font', 'Wilmer', 2018, 'R', 55.4, 3.5, 0, 24.8, 12.4, 0, 3.9, 0, 'LAD',33.2+10.1+6.2,  1.67+13.18, 3.43+2.72,0,0]
Nuno_2 = ['Nuno', 'Vidal', 2017, 'L', 27.9, 52.5, 6.6, 7.5, 5.6, 0, 0, 0, 'BAL', 14.2, 1.64+8.79, 3.5737+1.7854, 0,0   ]

# Create new DataFrame from data
new_rows = pd.DataFrame([Cedeno_2, Romo_2, Cishek_2, Font_2, Nuno_2], columns=merged_df.columns)

# Concatenate merged_df and the new rows
merged_df = pd.concat([merged_df, new_rows], ignore_index=True)
merged_df.head(4)

,last_name,first_name,year,pitch_hand,ff_pct,sl_pct,ch_pct,cu_pct,si_pct,fc_pct,fs_pct,kn_pct,teamID,InningsPitched,ERA,FIP,ERA_diff,FIP_diff
0,Font,Wilmer,2018,R,59.1,9.3,0.0,17.0,6.4,0.0,8.2,0.0,TBA,27.0000,1.67,3.4346,-13.18,-2.7154
1,Nuno,Vidal,2018,L,17.4,67.0,9.2,6.4,0.0,0.0,0.0,0.0,TBA,33.0000,1.64,3.5737,-8.79,-1.7854
2,Cedeno,Xavier,2015,L,44.0,0.0,1.6,52.5,1.8,0.0,0.0,0.0,TBA,43.0000,2.09,3.1147,-3.91,-1.3075
3,Romo,Sergio,2017,R,18.7,58.4,7.2,0.0,15.7,0.0,0.0,0.0,TBA,30.6667,1.47,3.1022,-4.65,-0.9645


In [32]:
#group rows together by player
grouped_order = [8,0,9,1,5,2,6,3,7,4]
merged_df = merged_df.iloc[grouped_order]
merged_df

,last_name,first_name,year,pitch_hand,ff_pct,sl_pct,ch_pct,cu_pct,si_pct,fc_pct,fs_pct,kn_pct,teamID,InningsPitched,ERA,FIP,ERA_diff,FIP_diff
8,Font,Wilmer,2018,R,55.4,3.5,0.0,24.8,12.4,0.0,3.9,0.0,LAD,49.5000,14.85,6.1500,0.00,0.0000
0,Font,Wilmer,2018,R,59.1,9.3,0.0,17.0,6.4,0.0,8.2,0.0,TBA,27.0000,1.67,3.4346,-13.18,-2.7154
9,Nuno,Vidal,2017,L,27.9,52.5,6.6,7.5,5.6,0.0,0.0,0.0,BAL,14.2000,10.43,5.3591,0.00,0.0000
1,Nuno,Vidal,2018,L,17.4,67.0,9.2,6.4,0.0,0.0,0.0,0.0,TBA,33.0000,1.64,3.5737,-8.79,-1.7854
5,Cedeno,Xavier,2014,L,59.8,0.0,1.8,32.1,6.3,0.0,0.0,0.0,WSH,10.0000,4.50,7.4600,0.00,0.0000
2,Cedeno,Xavier,2015,L,44.0,0.0,1.6,52.5,1.8,0.0,0.0,0.0,TBA,43.0000,2.09,3.1147,-3.91,-1.3075
6,Romo,Sergio,2016,R,7.9,63.5,7.3,0.0,21.2,0.0,0.0,0.0,SFG,30.2000,6.12,4.0700,0.00,0.0000
3,Romo,Sergio,2017,R,18.7,58.4,7.2,0.0,15.7,0.0,0.0,0.0,TBA,30.6667,1.47,3.1022,-4.65,-0.9645
7,Cishek,Steve,2016,R,1.5,50.1,1.0,0.0,47.3,0.0,0.0,0.0,SEA,64.0000,3.15,3.7000,0.00,0.0000
4,Cishek,Steve,2017,R,2.2,48.7,1.4,0.0,47.7,0.0,0.0,0.0,TBA,24.6667,1.09,2.7811,-2.06,-0.9189


In [18]:
#Export df for R visualization
merged_df.to_csv('5ImprovedRaysPitchersV2.csv', index_label=False)